# Imports
Import python libraries

In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from itertools import combinations, product

# Set dataset parameters
Choose the attributes to load the dataset.

In [2]:


detector = 'mtcnn_serfiq'  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_model = 'QMagFace'  # 'QMagFace', 'QMagFace_SR',"ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'

#quality_models = ['ser_fiq'] #  'ser_fiq','tface'
#embedding_models = ['Facenet512','QMagFace','ArcFace'] 
#quality_models = ['ser_fiq','tface']

# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [3]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'

#calibration_db0 = ['XQLFW','SCFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',
#image_filters0 = ['yaw', 'gender', 'race','emotion']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion','race'

calibration_db = ['XQLFW'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW','UTKface','ENFSI',
#'XQLFW','UTKface',"SCFace",'ForenFace',



In [4]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Input and output directories
Where the sql database is and which folder the results are saved.

In [5]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset_21b'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'experiment')

# Preprocess data
Connect to sql database.

In [6]:
from sql_face.alchemy import get_session
from sql_face.tables import *

2023-07-01 16:02:30.963783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
session = get_session(input_dir, db_name)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [9]:
results1 = session.query(Image,FaceImage) \
.select_from(Image).filter(Image.source == 'XQLFW') \
.join(CroppedImage).join(Detector).filter(Detector.name == detector) \
.join(FaceImage).join(EmbeddingModel).filter(EmbeddingModel.name == embedding_model).all()



In [10]:
session.close()

Creo el dataframe con los resultados

In [11]:
im_cols = ['identity',  'path' , 'image_id']
fi_cols = ['embeddings']
#fixed_cols = ['Detector','E_Model','Q_Model','Q_group','Quality']

df1 = pd.DataFrame([ [getattr(r[0],attr)   for attr in im_cols]  + [r[1].embeddings]
                   for r in results1],
                   columns=im_cols + fi_cols )

In [12]:
if len(df1):
    df1.to_pickle(os.path.join(output_dir,'imagenes.pkl'))

In [13]:
identities = set(df1.identity)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/jmacarulla/anaconda3/envs/lr-video/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/jmacarulla/anaconda3/envs/lr-video/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

Defino la funcion de distancia o similitud

In [ ]:
#hago la funcion coseno por hacer una
def similitud(a,b):
    a1 = np.array(a)
    b1 = np.array(b)

    return( np.dot(a1,b1)/(np.linalg.norm(a1)*np.linalg.norm(b1)))


Buscar dentro de la misma identidad

In [ ]:
df_resultado = pd.DataFrame(columns=['path1', 'path2', 'resultado'])

In [ ]:
#identities = set(df1.identity)
identities = []

In [ ]:

for identity in tqdm(identities) :
    dfx = df1.loc[(df1.identity == identity)]
    if len(dfx) > 1:
        combinaciones = list(combinations(dfx.iterrows(), 2))

        # Iterar sobre las combinaciones
        for comb in tqdm(combinaciones):
            index1, row1 = comb[0]
            index2, row2 = comb[1]
    
            path1 = row1['path']
            path2 = row2['path']
    
            # Realizar la operación deseada (ejemplo: suma de los embeddings)
            resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
            df_resultado = df_resultado.append({
            'path1': path1,
            'path2': path2,
            'resultado': resultado
            }, ignore_index=True)
        
        


In [ ]:
if len(df_resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'misma_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'misma_identidad.xlsx'))

Buscar en diferentes identidades

In [ ]:
identities = list(set(df1.identity))
#identities = []

In [ ]:
df_resultado = pd.DataFrame(columns=['path1', 'path2', 'resultado'])

In [ ]:
for i,identity in enumerate(identities):
    print(i)
    dfx = df1.loc[(df1.identity == identity)]
    dfy = df1.loc[(df1.identity > identity)]
    
    combinaciones = list(product(dfx.iterrows(), dfy.iterrows()))

        # Iterar sobre las combinaciones
    df_parcial = pd.DataFrame(columns=['path1', 'path2', 'resultado'])
    for comb in tqdm(combinaciones):
        
        index1, row1 = comb[0]
        index2, row2 = comb[1]
    
        path1 = row1['path']
        path2 = row2['path']
    
        # Realizar la operación deseada (ejemplo: suma de los embeddings)
        resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
        df_parcial = df_parcial.append({
        'path1': path1,
        'path2': path2,
        'resultado': resultado
        }, ignore_index=True)

        if len(df_parcial)>1000:
            df_ordenado = df_parcial.sort_values(by='resultado', ascending=False)
            df_parcial = df_ordenado.head(10)

    #para cada identidad solo cogemos las 10 imágenes que más se le parecen     
    df_ordenado = df_parcial.sort_values(by='resultado', ascending=False)
    df_resultado = df_resultado.append(df_ordenado.head(10), ignore_index=True)
    df_resultado.to_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))
        

In [23]:
df_resultado.to_excel(os.path.join(output_dir,'diferente_identidad_0.xlsx'))

In [ ]:
df_resultado = pd.read_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))

In [ ]:
if len(df_resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'diferente_identidad.xlsx'))

In [ ]:
len(identities)